In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install chromadb huggingface_hub langchain
!pip install -U langchain-community
!pip install sentence-transformers
!pip install langchain requests
!pip install langdetect
!pip install beautifulsoup4
!pip install langchain together
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.2/159.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 3.4 MB/s eta 0:00:0

In [ ]:
import os
# Set the USER_AGENT environment variable
os.environ['USER_AGENT'] = 'MyAppName/1.0'

In [ ]:
# Import the necessary classes
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
import requests
import re
import os
import numpy as np
import pandas as pd

In [ ]:
# Helper function to load or create a Chroma database
def create_or_load_chroma_db(persist_directory, embedding_model):
    if not os.path.exists(persist_directory):
        os.makedirs(persist_directory)
        print(f"Created directory for Chroma database at: {persist_directory}")

    try:
        # Load the existing ChromaDB
        chroma_db = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)
        print(f"Loaded existing Chroma database from {persist_directory}")
    except Exception as e:
        print(f"Error loading Chroma database: {e}")
        # If loading fails, create a new Chroma database
        print(f"Creating a new Chroma database at {persist_directory}")
        try:
            chroma_db = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)
            print("New Chroma database created successfully.")
        except Exception as e:
            print(f"Error creating Chroma database: {e}")
            raise  # Re-raise the exception after logging

    return chroma_db

In [ ]:
def create_retriever(chroma_db, search_type="similarity", threshold=0.55, k=4, lambda_mult=0.25):
    retriever = chroma_db.as_retriever(
        search_type=search_type,
        relevance_score_threshold=threshold,
        k=k,
        lambda_mult=lambda_mult
    )
    return retriever

In [ ]:
# Load the Hugging Face embedding model
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
chroma_db_path ="/content/drive/MyDrive/rag-model/ChromaDB_Lang_Chain"
chroma_db_drive=create_or_load_chroma_db(chroma_db_path,embedding_model)

<ipython-input-7-e8f9ff2065ec>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your se

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-5-83189448e33f>:9: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_db = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)


Loaded existing Chroma database from /content/drive/MyDrive/rag-model/ChromaDB_Lang_Chain


In [ ]:
# Clean HTML from retrieved answers
def clean_and_format_answers(answers):
    clean_html = re.compile('<.*?>')
    cleaned_text = ""
    for answer in answers:
        cleaned_text += f"Question: {answer['question']}\n\n"
        for doc in answer['relevant_documents']:
            cleaned_doc = re.sub(clean_html, '', doc)  # Clean HTML tags
            cleaned_text += f"{cleaned_doc.strip()}\n\n"
    return cleaned_text.strip()

In [ ]:
# Function to retrieve documents based on questions
def get_answers(questions, retriever):
    answers = []
    for question in questions:
        results = retriever.get_relevant_documents(question)
        relevant_docs = [result.page_content for result in results]
        answers.append({
            "question": question,
            "relevant_documents": relevant_docs
        })
    return answers

In [ ]:
# Function to generate embeddings for a list of texts
def get_embeddings(texts, embedding_model):
    return embedding_model.embed_documents(texts)

In [ ]:
# Function to calculate Euclidean distance
def calculate_euclidean_distance(query_embedding, doc_embeddings):
    # Convert to NumPy arrays if they are lists
    query_embedding = np.array(query_embedding)
    doc_embeddings = np.array(doc_embeddings)

    # Calculate distances
    distances = np.linalg.norm(doc_embeddings - query_embedding, axis=1)
    return distances

In [ ]:
# Function to calculate Manhattan distance
def calculate_manhattan_distance(query_embedding, doc_embeddings):
    # Convert to NumPy arrays if they are not already
    query_embedding = np.array(query_embedding)
    doc_embeddings = np.array(doc_embeddings)

    # Calculate Manhattan distances
    distances = np.abs(doc_embeddings - query_embedding).sum(axis=1)
    return distances

In [ ]:
# Function to calculate dot product
def calculate_dot_product(query_embedding, doc_embeddings):
    return np.dot(doc_embeddings, query_embedding)

In [ ]:
# Function to calculate cosine similarity for embeddings
def calculate_cosine_similarity(query_embedding, doc_embeddings):
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
    return similarities

In [ ]:
# Function to calculate similarities between all document embeddings
def calculate_document_similarities(doc_embeddings, metric='cosine'):
    # Prepare an array to store similarity scores
    num_docs = len(doc_embeddings)
    similarity_matrix = np.zeros((num_docs, num_docs))

    for i in range(num_docs):
        for j in range(num_docs):
            if metric == 'cosine':
                similarity_matrix[i][j] = cosine_similarity([doc_embeddings[i]], [doc_embeddings[j]])[0][0]
            elif metric == 'euclidean':
                similarity_matrix[i][j] = euclidean_distances([doc_embeddings[i]], [doc_embeddings[j]])[0][0]
            elif metric == 'manhattan':
                similarity_matrix[i][j] = manhattan_distances([doc_embeddings[i]], [doc_embeddings[j]])[0][0]
            elif metric == 'dot':
                similarity_matrix[i][j] = np.dot(doc_embeddings[i], doc_embeddings[j])
            else:
                raise ValueError("Unknown metric: choose from 'cosine', 'euclidean', 'manhattan', or 'dot'.")

    return similarity_matrix

In [ ]:
# Main function to load embeddings, retrieve documents, and calculate similarities..
if __name__ == "__main__":
    # Path for Chroma database
    chroma_db_path = "/content/drive/MyDrive/rag-model/ChromaDB_Lang_Chain"

    # Create or load Chroma database
    chroma_db = create_or_load_chroma_db(chroma_db_path, embedding_model)

    # Create a retriever
    retriever = create_retriever(chroma_db=chroma_db)

    # Define the question
    questions = [
        "How does the ASEAN Human Rights Declaration emphasize the universality, indivisibility, interdependence, and interrelatedness of human rights?"
    ]

    # Retrieve relevant answers
    answers = get_answers(questions, retriever)

    for answer in answers:
        # Generate question embedding
        question_embedding = np.array(embedding_model.embed_query(answer['question']))

        # Generate document embeddings
        doc_embeddings = np.array(get_embeddings(answer['relevant_documents'], embedding_model))

        # Calculate similarities and distances for each document to question
        cosine_similarities = calculate_cosine_similarity(question_embedding, doc_embeddings)
        euclidean_dists = calculate_euclidean_distance(question_embedding, doc_embeddings)
        manhattan_dists = calculate_manhattan_distance(question_embedding, doc_embeddings)
        dot_products = calculate_dot_product(question_embedding, doc_embeddings)

        # Display document-to-question similarity metrics
        print(f"Question: {answer['question']}\n")
        for i, doc in enumerate(answer['relevant_documents']):
            print(f"Document {i+1}:\n"
                  #f"Document: {doc}\n"
                  f"Cosine Similarity: {cosine_similarities[i]:.4f}\n"
                  f"Euclidean Distance: {euclidean_dists[i]:.4f}\n"
                  f"Manhattan Distance: {manhattan_dists[i]:.4f}\n"
                  f"Dot Product: {dot_products[i]:.4f}\n")

        # Calculate document-to-document similarities
        document_similarity_matrix = calculate_document_similarities(doc_embeddings, metric='cosine')

        # Convert the similarity matrix to a DataFrame with document labels
        doc_labels = [f"Document {i+1}" for i in range(len(doc_embeddings))]
        similarity_df = pd.DataFrame(document_similarity_matrix, index=doc_labels, columns=doc_labels)

        # Display the labeled document similarity matrix
        print("\nDocument-to-Document Similarity Matrix (Cosine):")
        print(similarity_df)

Loaded existing Chroma database from /content/drive/MyDrive/rag-model/ChromaDB_Lang_Chain


<ipython-input-9-a2f5575faf9b>:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(question)


Question: How does the ASEAN Human Rights Declaration emphasize the universality, indivisibility, interdependence, and interrelatedness of human rights?

Document 1:
Cosine Similarity: 0.5342
Euclidean Distance: 0.9652
Manhattan Distance: 21.0647
Dot Product: 0.5342

Document 2:
Cosine Similarity: 0.5138
Euclidean Distance: 0.9861
Manhattan Distance: 21.3936
Dot Product: 0.5138

Document 3:
Cosine Similarity: 0.5769
Euclidean Distance: 0.9199
Manhattan Distance: 20.3536
Dot Product: 0.5769

Document 4:
Cosine Similarity: 0.5963
Euclidean Distance: 0.8985
Manhattan Distance: 19.6221
Dot Product: 0.5963


Document-to-Document Similarity Matrix (Cosine):
            Document 1  Document 2  Document 3  Document 4
Document 1    1.000000    0.793384    0.632947    0.666458
Document 2    0.793384    1.000000    0.630330    0.677724
Document 3    0.632947    0.630330    1.000000    0.751039
Document 4    0.666458    0.677724    0.751039    1.000000
